In [2]:
import pandas as pd
import json
df = pd.read_csv('nba_alltime_leaders.csv')
cols = ['PLAYER', 'GP', 'PTS', 'FT%', 'FTM', 'FTA', 'FGM','FGA','FG%','3PM','3PA','3P%','OREB','DREB','REB','AST','STL','BLK','TOV']
data = df[cols].sort_values("FT%", ascending=False).drop_duplicates(subset=['PLAYER'])
data = data.reset_index()[cols]
data_dict = data.to_dict(orient='records')
with open('players_ft.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)




In [34]:
result = []
for key, value in data.iterrows():
    d = {}
    #print(value['PLAYER'])
    #print(value['FT%'])
    d['Player'] = value['PLAYER']
    d['ft%'] = value['FT%']
    result.append(d)

In [35]:
result

[{'Player': 'Stephen Curry', 'ft%': 91.0},
 {'Player': 'Damian Lillard', 'ft%': 89.7},
 {'Player': 'Kyrie Irving', 'ft%': 88.6},
 {'Player': 'Kevin Durant', 'ft%': 88.4},
 {'Player': 'Desmond Bane', 'ft%': 87.9},
 {'Player': 'Khris Middleton', 'ft%': 87.9},
 {'Player': 'Jordan Poole', 'ft%': 87.8},
 {'Player': 'Danilo Gallinari', 'ft%': 87.6},
 {'Player': 'Malcolm Brogdon', 'ft%': 87.3},
 {'Player': 'Anfernee Simons', 'ft%': 87.3},
 {'Player': 'Trae Young', 'ft%': 87.3},
 {'Player': 'Isaiah Thomas', 'ft%': 87.2},
 {'Player': 'Devin Booker', 'ft%': 87.0},
 {'Player': 'Chris Paul', 'ft%': 87.0},
 {'Player': 'CJ McCollum', 'ft%': 86.9},
 {'Player': 'Kawhi Leonard', 'ft%': 86.2},
 {'Player': 'James Harden', 'ft%': 86.1},
 {'Player': 'Bojan Bogdanovic', 'ft%': 85.9},
 {'Player': 'Klay Thompson', 'ft%': 85.8},
 {'Player': 'Jeremy Lamb', 'ft%': 85.7},
 {'Player': 'Tyrese Haliburton', 'ft%': 85.6},
 {'Player': 'Austin Reaves', 'ft%': 85.5},
 {'Player': 'Cam Thomas', 'ft%': 85.5},
 {'Player': '

In [10]:
import json
# Replace 'your_file.json' with the path to your JSON file
with open('players_ft.json', 'r') as file:
    data = json.load(file)

d = {}
for p in data:
    d[p['PLAYER']] =  p['id']
with open('playerIndex.json', 'w') as file:
    json.dump(d, file, indent=4)


In [16]:
with open('playerIndex.json', 'r') as file:
    playerIndex = json.load(file)

with open('players_ft.json', 'r') as file:
    playerData = json.load(file)
for player in playerData:
    tempPlayer = player['PLAYER']
    if tempPlayer in playerIndex:
        player['index'] = playerIndex[tempPlayer]

In [20]:
with open('playerData.json', 'w') as json_file:
    json.dump(playerData, json_file, indent=4)

In [24]:
# Converts the JSON to SQLite database
import json
import sqlite3

# Load JSON data
with open('nbaPlayerData.json', 'r') as f:
    players_data = json.load(f)

# Connect to SQLite database (or create it)
conn = sqlite3.connect('nbaPlayers.db')
cursor = conn.cursor()

# Create the table with "index" quoted
cursor.execute('''
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY,
    PLAYER TEXT,
    GP INTEGER,
    PTS INTEGER,
    FT_Percentage REAL,
    FTM INTEGER,
    FTA INTEGER,
    FGM INTEGER,
    FGA INTEGER,
    FG_Percentage REAL,
    ThreePM INTEGER,
    ThreePA INTEGER,
    ThreeP_Percentage REAL,
    OREB INTEGER,
    DREB INTEGER,
    REB INTEGER,
    AST INTEGER,
    STL INTEGER,
    BLK INTEGER,
    TOV INTEGER,
    PICIDX INTEGER
)
''')

# Insert data into the table
for player in players_data:
    cursor.execute('''
    INSERT INTO players (
        PLAYER, GP, PTS, FT_Percentage, FTM, FTA, FGM, FGA, FG_Percentage, 
        ThreePM, ThreePA, ThreeP_Percentage, OREB, DREB, REB, AST, STL, BLK, TOV, "index"
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        player['PLAYER'], player['GP'], player['PTS'], player['FT%'], player['FTM'], 
        player['FTA'], player['FGM'], player['FGA'], player['FG%'], player['3PM'], 
        player['3PA'], player['3P%'], player['OREB'], player['DREB'], player['REB'], 
        player['AST'], player['STL'], player['BLK'], player['TOV'], player['index']
    ))
    

# Commit changes and close the connection
conn.commit()
conn.close()

print("Data inserted successfully!")


Data inserted successfully!


In [22]:
with open('tempPlayerData.json', 'r') as f:
    players_data = json.load(f)
new_players = []
for player in players_data:
    if 'index' in player:
        new_players.append(player)

with open('nbaPlayerData.json', 'w') as file:
    json.dump(new_players, file, indent=4)

In [25]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('nbaPlayers.db')
cursor = conn.cursor()

# Query to fetch all data from the players table
cursor.execute('SELECT * FROM players')

# Fetch all rows
rows = cursor.fetchall()

# Print the column names
column_names = [description[0] for description in cursor.description]
print("Column Names:", column_names)

# Print each row
for row in rows:
    print(row)

# Close the connection
conn.close()


Column Names: ['id', 'PLAYER', 'GP', 'PTS', 'FT_Percentage', 'FTM', 'FTA', 'FGM', 'FGA', 'FG_Percentage', 'ThreePM', 'ThreePA', 'ThreeP_Percentage', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'index']
(1, 'Stephen Curry', 956, 23668, 91.0, 3753, 4125, 8084, 17098, 47.3, 3747, 8805, 42.6, 629, 3880, 4509, 6119, 1473, 235, 2987, 201939)
(2, 'Damian Lillard', 842, 21151, 89.7, 4900, 5460, 6822, 15576, 43.8, 2607, 7032, 37.1, 505, 3068, 3573, 5659, 804, 257, 2379, 203081)
(3, 'Reggie Miller', 144, 2972, 89.3, 770, 862, 941, 2095, 44.9, 320, 820, 39.0, 77, 339, 416, 357, 146, 34, 257, 397)
(4, 'Dirk Nowitzki', 145, 3663, 89.2, 1074, 1204, 1220, 2642, 46.2, 149, 408, 36.5, 212, 1234, 1446, 360, 149, 129, 328, 1717)
(5, 'Larry Bird', 164, 3897, 89.0, 901, 1012, 1458, 3090, 47.2, 80, 249, 32.1, 360, 1323, 1683, 1062, 296, 145, 506, 1449)
(6, 'Kyrie Irving', 729, 17199, 88.6, 2787, 3146, 6340, 13368, 47.4, 1732, 4404, 39.3, 578, 2355, 2933, 4134, 956, 314, 1831, 202681)
(7, 'Kevin Duran

In [ ]:
# Importing new data for leaderbaord
import pandas as pd

pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_totals.html#totals_stats')

[        Rk                 Player   Age Team  Pos     G    GS      MP     FG  \
 0      1.0           James Harden  30.0  HOU   SG  68.0  68.0  2483.0  672.0   
 1      2.0         Damian Lillard  29.0  POR   PG  66.0  66.0  2474.0  624.0   
 2      3.0           Devin Booker  23.0  PHO   SG  70.0  70.0  2512.0  627.0   
 3      4.0  Giannis Antetokounmpo  25.0  MIL   PF  63.0  63.0  1917.0  685.0   
 4      5.0             Trae Young  21.0  ATL   PG  60.0  60.0  2120.0  546.0   
 ..     ...                    ...   ...  ...  ...   ...   ...     ...    ...   
 647  648.0         William Howard  26.0  HOU   PF   2.0   0.0    13.0    0.0   
 648  649.0           Stanton Kidd  27.0  UTA   SG   4.0   0.0    15.0    0.0   
 649  650.0            J.P. Macura  24.0  CLE   SG   1.0   0.0     1.0    0.0   
 650  651.0            Isaiah Roby  21.0  OKC   PF   3.0   0.0    11.0    0.0   
 651    NaN         League Average   NaN  NaN  NaN   NaN   NaN     NaN    NaN   
 
         FGA  ...    DRB  

In [22]:
import pandas as pd
url_attempt = "https://www.basketball-reference.com/leaders/fta_career.html"
df_attempt = pd.read_html(url_attempt)[1]
url_made = "https://www.basketball-reference.com/leaders/ft_career.html"
df_made = pd.read_html(url_made)[1]
df = pd.merge(df_made, df_attempt, on='Player')
cols = ['Player', 'FT', 'FTA']
df = df[cols]
df['Percentage'] = df['FT'] / df['FTA']
df = df.sort_values(by='Percentage', ascending=False)
df = df.reset_index()
cols2 = ['Player', 'FT', 'FTA', 'Percentage']
df = df[cols2]
df["Player"] = df["Player"].str.replace(r"\*$", "", regex=True)

In [23]:
import json

with open('playerIndex.json', 'r') as file:
    playerIndexData = json.load(file)

import pandas as pd

dfPlayerIds = pd.DataFrame(
    list(playerIndexData.items()),
    columns=["Player", "Id"]
)
dfPlayerIds

,Player,Id
0,Stephen Curry,201939
1,Jared Mccain,1642272
2,Damian Lillard,203081
3,Reggie Miller,397
4,Dirk Nowitzki,1717
...,...,...
143,Kevin Garnett,708
144,Chris Bosh,2547
145,Jalen Rose,147
146,Joe Johnson,2207


In [24]:
final_df = pd.merge(df, dfPlayerIds, on='Player')
final_df

,Player,FT,FTA,Percentage,Id
0,Stephen Curry,4098,4494,0.911883,201939
1,Steve Nash,3060,3384,0.904255,959
2,Damian Lillard,5262,5853,0.899026,203081
3,Chauncey Billups,4496,5029,0.894015,1497
4,Ray Allen,4398,4920,0.893902,951
...,...,...,...,...,...
82,Rudy Gobert,2671,4176,0.639607,203497
83,Dwight Howard,5361,9455,0.567002,2730
84,Bill Russell,3148,5614,0.560741,78049
85,Shaquille O'Neal,5935,11252,0.527462,406


In [ ]:
# from nba_api.stats.endpoints import playercareerstats

# # Nikola Jokić
# career = playercareerstats.PlayerCareerStats(player_id='2544')

# # pandas data frames (optional: pip install pandas)
# df_regular_season = career.season_totals_regular_season.get_data_frame()

# df_post_season = career.career_totals_post_season.get_data_frame()
# # # json
# json_version = career.get_json()

# # # dictionary
# dictionary = career.get_dict()

In [ ]:

# import json
# import time
# import requests
# from nba_api.stats.endpoints import playercareerstats

# # Containers for JSON data
# regular_season_data = {}
# post_season_data = {}

# def get_player_stats(pid, retries=3, delay=2):
#     """
#     Fetch player career stats with retry logic.
#     """
#     for attempt in range(retries):
#         try:
#             return playercareerstats.PlayerCareerStats(player_id=pid)
#         except requests.exceptions.ReadTimeout:
#             print(f"Timeout for {pid}, retry {attempt+1}/{retries}...")
#             time.sleep(delay)
#         except Exception as e:
#             print(f"Error for {pid}: {e}")
#             return None
#     return None

# for pid in final_df['Id'].to_list():
#     career = get_player_stats(pid)
#     if career is None:
#         continue  # skip if failed after retries
    
#     # Regular season
#     df_regular = career.season_totals_regular_season.get_data_frame()
#     regular_season_data[pid] = df_regular.to_dict(orient="records")
    
#     # Postseason
#     df_post = career.season_totals_post_season.get_data_frame()
#     post_season_data[pid] = df_post.to_dict(orient="records")
    
#     # Sleep to avoid hitting rate limits
#     time.sleep(0.6)

# # Save to JSON files
# with open("regular_season2.json", "w") as f:
#     json.dump(regular_season_data, f, indent=2)

# with open("post_season2.json", "w") as f:
#     json.dump(post_season_data, f, indent=2)

# print("Data saved to regular_season2.json and post_season2.json")


Data saved to regular_season2.json and post_season2.json


In [52]:
final_df

,index,Player,FT,FTA,Percentage,Id
0,0,Stephen Curry,4098,4494,0.911883,201939
1,1,Steve Nash,3060,3384,0.904255,959
2,2,Damian Lillard,5262,5853,0.899026,203081
3,3,Chauncey Billups,4496,5029,0.894015,1497
4,4,Ray Allen,4398,4920,0.893902,951
...,...,...,...,...,...,...
82,82,Rudy Gobert,2671,4176,0.639607,203497
83,83,Dwight Howard,5361,9455,0.567002,2730
84,84,Bill Russell,3148,5614,0.560741,78049
85,85,Shaquille O'Neal,5935,11252,0.527462,406


In [46]:
final_df.to_json('playersFreeThrowStatistics.json')